Import arcpy, ustawienie środowiska projektu

In [84]:
import arcpy

arcpy.env.workspace = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdp")
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
arcpy.env.overwriteOutput = True
arcpy.env.extent = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer")
arcpy.env.mask = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer.shp")
arcpy.env.cellSize = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\raster_extracted.tif")

Wczytanie warstw i ich zapis do zmiennych

In [59]:
# wylaczenie dodawania warstw
# arcpy.env.addOutputsToMap = False
drogi_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\SKDR.shp")
rzeki = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\SWRS.shp")
jeziora = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\PTWP.shp")
lasy = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\PTLZ.shp")
budynki_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\BUBDA.shp")
bufor = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer")
raster_gmina = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\raster_extracted")

Selekcja po atrybutach odpowiednich warstw

In [69]:
#wyselekcjonowanie drog utwardzonych
drogi = arcpy.management.SelectLayerByAttribute(drogi_wszystkie, "NEW_SELECTION", "MATE_NAWIE IN ('beton','bruk', 'kostka kamienna', 'kostka prefabrykowana', 'masa bitumiczna', 'płyty betonowe')", None)
arcpy.management.CopyFeatures(drogi, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\drogi_selected", '', None, None, None)
drogi_selected = 'drogi_selected'
arcpy.SelectLayerByAttribute_management(drogi, "CLEAR_SELECTION")

#wyselekcjonowanie budynkow mieszkalnych
budynki = arcpy.management.SelectLayerByAttribute(budynki_wszystkie, "NEW_SELECTION", "FOBUD IN ('budynki mieszkalne')", None)
arcpy.management.CopyFeatures(budynki, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\budynki_selected", '', None, None, None)
budynki_selected = 'budynki_selected'
arcpy.SelectLayerByAttribute_management(budynki, "CLEAR_SELECTION")

<Result 'BUBDA_Layer1'>

Stworzenie map odległości dla kryteriów narzędziem Euclidian Distance

In [70]:
distance_rzeki = arcpy.sa.EucDistance(rzeki, None, "5")
distance_jeziora = arcpy.sa.EucDistance(jeziora, None, "5")
distance_drogi = arcpy.sa.EucDistance(drogi_selected, None, "5")
distance_budynki = arcpy.sa.EucDistance(budynki_selected, None, "5")
distance_lasy = arcpy.sa.EucDistance(lasy, None, "5")

Kryterium 1 - odległość od rzek i zbiorników wodnych, jak najbliżej (nieprzekraczalna strefa ochronna 100m)

In [33]:
#kryterium ostre dla strefy ochronnej 100m poprzez reklasyfikacje map odleglosci
rzeki_ostre = arcpy.ddd.Reclassify(distance_rzeki, "VALUE", "0 100 0;100 912,537659 1")
jeziora_ostre = arcpy.ddd.Reclassify(distance_jeziora, "VALUE", "0 100 0;100 1773,760132 1")

#połączenie kryteriów ostrych dla rzek i jezior
wody_ostre = arcpy.sa.FuzzyOverlay([rzeki_ostre, jeziora_ostre], "AND")

In [35]:
#kryterium rozmyte dla obszarów w odległości powyżej 100m
rzeki_rozmyte = arcpy.sa.FuzzyMembership(distance_rzeki, arcpy.sa.FuzzyLinear(400, 200))
jeziora_rozmyte = arcpy.sa.FuzzyMembership(distance_jeziora, arcpy.sa.FuzzyLinear(400, 200))

#połączenie kryteriów rozmytych dla rzek i jezior
wody_rozmyte = arcpy.sa.FuzzyOverlay([rzeki_rozmyte, jeziora_rozmyte], "OR")

In [48]:
#połączenie kryterium ostrego i rozmytego dla wody
kryterium_wody = arcpy.sa.FuzzyOverlay([wody_ostre, wody_rozmyte], 'AND')
kryterium_wody.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wody")

Kryterium 2 - dostęp do dróg utwardzonych, jak największe zagęszczenie 

In [87]:
#kryterium rozmyte - do 200m najlepsza przydatność
kryterium_drogi = arcpy.sa.FuzzyMembership(distance_drogi, arcpy.sa.FuzzyLinear(500, 200))
kryterium_drogi.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_drogi")

Kryterium 3 - odległość od budynków mieszkalnych

In [77]:
kryterium_budynki = arcpy.sa.FuzzyMembership(distance_budynki,  arcpy.sa.FuzzyLinear(150, 400))
kryterium_budynki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_budynki")

Krterium 4 - pokrycie terenu, nie w lesie, powyżej 15m od lasu, optymalnie 100m od lasu 

In [88]:
kryterium_lasy = arcpy.sa.FuzzyMembership(distance_lasy, arcpy.sa.FuzzyLinear(15, 100))
kryterium_lasy.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_lasy")

Kryterium 5 - nachylenie stoków, optymalnie płasko, maksymalnie 10%

In [68]:
#stworzenie warstwy z nachyleniami stoków narzędziem Slope
spadki_output = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\spadki"
spadki = arcpy.ddd.Slope(raster_gmina, spadki_output, "PERCENT_RISE", 1)

kryterium_spadki = arcpy.sa.FuzzyMembership(spadki, arcpy.sa.FuzzyLinear(10, 3))
kryterium_spadki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_spadki")

Kryterium 6 - nasłonecznienie, optymalnie stoki południowe (SW-SE)

In [83]:
#stworzenie warstwy ze stokami narzędziem Aspect
stoki_output = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\stoki"
stoki = arcpy.ddd.Aspect(raster_gmina, stoki_output)
stoki_ES = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(95, 113))
stoki_WS = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(260, 247))
stoki_ostre = arcpy.ddd.Reclassify(stoki, "VALUE", "0 113 0;113 247 1;247 360 0")